In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from sklearn.preprocessing import StandardScaler
import tensorflow as tf 
from tensorflow import keras
import matplotlib.pyplot as plt 
%matplotlib inline

In [ ]:
x_train = pd.read_csv('./temp_train.csv').to_numpy()
y_train = pd.read_csv('./htc_train.csv').to_numpy()

In [ ]:
x_valid = pd.read_csv('./temp_valid.csv').to_numpy()
y_valid = pd.read_csv('./htc_valid.csv').to_numpy()

In [ ]:
x_test = pd.read_csv('./temp_test.csv').to_numpy()
y_test = pd.read_csv('./htc_test.csv').to_numpy()

In [ ]:
scaler = StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_valid = scaler.transform(x_valid)
x_test = scaler.transform(x_test)

In [ ]:
x_train_seq = x_train.reshape(-1, 120, 1)
x_valid_seq = x_valid.reshape(-1, 120, 1)
x_test_seq = x_test.reshape(-1, 120, 1)

In [ ]:
y_train_seq = y_train.reshape(-1, 120, 1)
y_valid_seq = y_valid.reshape(-1, 120, 1)
y_test_seq = y_test.reshape(-1, 120, 1)

In [ ]:
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, TimeDistributed

In [ ]:
model = tf.keras.Sequential()

# Add the bidirectional LSTM layers
model.add(Bidirectional(LSTM(50, return_sequences=True), input_shape=(120, 1)))
model.add(Bidirectional(LSTM(100, return_sequences=True)))
model.add(Bidirectional(LSTM(100, return_sequences=True)))
model.add(TimeDistributed(Dense(1)))

In [ ]:
model.compile(loss='mse', optimizer=keras.optimizers.Adam(learning_rate=0.001), metrics=[
    keras.metrics.RootMeanSquaredError(),
    keras.metrics.MeanAbsolutePercentageError()
])

In [ ]:
with tf.device('/GPU:0'):
    history = model.fit(x_train_seq, y_train_seq, epochs=10, validation_data=(x_valid_seq, y_valid_seq))

In [ ]:
model.save('./BiLSTM_.h5')

In [ ]:
model.evaluate(x_test_seq, y_test_seq)

In [ ]:
indx = 2006
x = (scaler.inverse_transform(x_train))[indx]
plt.plot(x, y_train_seq[indx], label='original')
plt.plot(x, model.predict(x_train_seq[indx:indx+1])[0][:, -1], label='predicted')
  
# Naming the x-axis, y-axis and the whole graph
plt.ylabel("HTC (W/m$^2$K)")
plt.title("HTC vs Temperature")
  
plt.legend()
  
plt.show()